# Задание 8

Даны результаты работы трех машинных переводчиков (N,P,Z) на небольших выборках переводов с английского на немецкий, с английского на русский и с казахского на русский

Требуется определить для каких пар языков можно выделить какого-либо лидера среди переводчиков. Для BLEU учитывать только слова, регистр не учитывать.

In [131]:
import pymorphy2
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate import bleu_score as score 
import re
import string

import os
import matplotlib.pyplot as plt
%matplotlib inline

import itertools
import numpy as np
import pandas as pd

import seaborn as sns

import scipy.stats as st

from statsmodels.stats.descriptivestats import sign_test

## Загружаем коллекцию документов как словарь
collection[название документа] = текст документа

In [257]:
def load_collection(path):
    collection = dict()
    for file_name in os.listdir(path):
        with open(path + '/' + file_name, 'r') as f_input:
            new_document = f_input.read()
            collection[file_name.strip(".txt")] = new_document
    return collection

collection = load_collection("./mt")

In [4]:
collection.keys()

dict_keys(['en_ru_n', 'en_de_n', 'kk_ru_orig', 'en_ru_z.txt~', 'kk_ru_n', 'kk_ru_gold', 'kk_ru_p', 'en_ru_gold', 'en_ru_p', 'en_ru_z', 'en_de_p', 'en_de_z', 'en_de_gold', 'en_ru_orig', 'kk_ru_z', 'en_de_orig'])

### Предобработка коллекции:
 - Текст бьется на предложения
 - Внутри предложения:
  - Слова приводятся к нижнему регистру
  - Убираются знаки препинания и цифры
  - Проводится токенезация
 
В результате получается, что каждый текст - список предложений, каждое предложение - список токенов.

In [5]:
def text_prepare(text):
    new_text = []
    sentences = text.split("\n")
    for i, sent in enumerate(sentences):
        if len(sent) > 0:
            sent = re.sub(r"["+ string.punctuation + "1234567890" + "]",  ' ', sent)
            sent = sent.lower()
            sent = re.sub(' +', ' ', sent)
            new_text.append(sent.strip().split(" "))
    return new_text

### Создаем новый словарь с предобработанной коллекцией

In [6]:
prepared_collection = dict()
for key in collection.keys():
    prepared_collection[key]= text_prepare(collection[key])
        

## Функция, которая для конкретной пары языков считает sentence_bleu_score для каждых трех переводчиков:

In [132]:
def calculate_score(lang_pair):
    score_n = []
    for i in range(len(prepared_collection[lang_pair + "_gold"])):
        score_n.append(sentence_bleu([prepared_collection[lang_pair + "_gold"][i]], \
                                   prepared_collection[lang_pair + "_n"][i],\
                                   smoothing_function = score.SmoothingFunction().method3))#, weights=(0.5, 0.25, 0.25)))
    score_z = []
    for i in range(len(prepared_collection[lang_pair + "_gold"])):
        score_z.append(sentence_bleu([prepared_collection[lang_pair + "_gold"][i]], \
                                   prepared_collection[lang_pair + "_z"][i],\
                                   smoothing_function = score.SmoothingFunction().method3))#, weights=(0.5, 0.25, 0.25)))

    score_p = []
    for i in range(len(prepared_collection[lang_pair + "_gold"])):
        score_p.append(sentence_bleu([prepared_collection[lang_pair + "_gold"][i]], \
                                   prepared_collection[lang_pair + "_p"][i], 
                                   smoothing_function = score.SmoothingFunction().method3))#, weights=(0.5, 0.25, 0.25)))
    return np.array(score_n), np.array(score_z), np.array(score_p)

### Проверка одиночной гипотезы
Мы видим, что выборки связанные и малеькие по размеру. 
Поэтому чтобы проверить одиночную гипотезу о среднем, мы можем использовать все те же критерии, что и в первом задании: Критерий знаков, перестановок, Уилкоксона и t-test, если Шапиро не отклоняет гипотезу о нормальности

Посмотрим что говорит нам критерий Шапиро о применимости t-testa:

Язык en-ru:

score_n, scor_z, score_p = calculate_score("en_ru")
print(st.shapiro(score_n))
print(st.shapiro(score_z))
print(st.shapiro(score_p))

Язык en_de:

In [206]:
score_n, scor_z, score_p = calculate_score("en_de")
print(st.shapiro(score_n))
print(st.shapiro(score_z))
print(st.shapiro(score_p))

(0.7521209716796875, 0.0037873859982937574)
(0.6457548141479492, 0.00019513994629960507)
(0.748779833316803, 0.003448653733357787)


язык kk_ru:

In [208]:
score_n, scor_z, score_p = calculate_score("kk_ru")
print(st.shapiro(score_n))
print(st.shapiro(score_z))
print(st.shapiro(score_p))

(0.6598911285400391, 0.00028866127831861377)
(0.6457548141479492, 0.00019513994629960507)
(0.8266522884368896, 0.030492015182971954)


#### Вывод: так как в большинстве случаев shapiro сильно отвергается, использовать ttest не будем
#### Кроме того, в текстах всего по 10 предложений - этого мало, чтобы проводить в Уилкоксоне нормальную аппроксимацию. Стандартную функцию не используем, используем свою функцию без аппроксимации

## Дизайн эксперимента №1

## Формулируем гипотезы:
Каждый тест мы разбиваем на предложения. Для каждого предложения считаем bleu_score. 
После этого для каждой пары языков и для каждого переводчика у нас появилась выборка $x_1, ..., x_n$ - blue_score предложений как оценка качества перевода. Для каждой пары языков и для каждого переводчика проводим множественное тестирование гипотез:
 - $H_{0_{Np}}: m_n = m_p$
 
   $H_{1_{Np}}: m_n < m_p$
   
и
   
 - $H_{0_{Nz}}: m_n = m_z$
 
   $H_{1_{Nz}}: m_n < m_z$ 
   
Если мы отвергли обе вышенаписанные гипотезы, мы можем сказать, что, например, N является лидером перевода для какой-то пары языков

### Множественное тестирование
Для множественного тестирования используем поправку Бенджамини — Хохберга

#### Функция проводит множественное тестирование с поправкой Бенджамини — Хохберга на уровне  $\alpha$ 

In [258]:
def calculate_Benjamini(leader, data1, data2, test, alpha, data_list):
    #data is shape set_number x set_size
    test_result = []
    if test.__name__ == 'permute_test':
        p_1 = tuple(test(leader, data1, "greater"))[1]
        p_2 = tuple(test(leader, data2, "greater"))[1]
    elif test.__name__ == "ttest_rel" :
        if st.shapiro(data1)[1] < alpha or st.shapiro(data2)[1] < alpha or st.shapiro(leader)[1] < alpha:
            print("Can't perform t-test")
            return 0
    else:
        p_1 = tuple(test(leader, data1))[1] 
        p_2 = tuple(test(leader, data2))[1]

    less, great = sorted([p_1, p_2])
    
    if less >= alpha/2:
        print("Can't reject both hypotheses, for [", data_list[0], ",", data_list[1], "] - ", p_1,\
                                             "for [", data_list[0], ",", data_list[2], "] - ", p_2)
    elif great >= alpha:
        print("One hypothesis rejected, for [", data_list[0], ",", data_list[1], "] - ", p_1,\
                                             "for [", data_list[0], ",", data_list[2], "] - ", p_2)
    else:
        print("Both hypotheses rejected, for [", data_list[0], ",", data_list[1], "] - ", p_1,\
                                             "for [", data_list[0], ",", data_list[2], "] - ", p_2)

In [259]:
def permute_test(data1, data2, alternative):
    D = data1 - data2
    t = np.sum(D)
    good = 0
    count = 0
    for v in list(itertools.product([-1, 1], repeat=data1.shape[0])):
        count +=1
        stat = np.array(v).T.dot(D)
        if alternative == "greater":
            if stat >= t:
                good+=1
        elif alternative == "lower":
            if stat <= t:
                good+=1
        elif alternative == "two-side":
            if abs(stat) >= abs(t):
                good+=1
        else:
            return Nan, Nan
    return t, good/count

In [251]:
def wilcoxon(data1, data2):
    rank = st.rankdata(np.abs(data1 - data2))
    sign = np.sign(data1 - data2)
    t = np.sum(rank*sign)
    good = 0
    count = 0
    for v in list(itertools.product([-1, 1], repeat=data1.shape[0])):
        count +=1
        stat = np.array(v).T.dot(rank)
        if abs(stat) >= abs(t):
            good +=1
    return t, good/count

In [252]:
def multitest(lang):
    #data has sahape num_translators x num_sentences
    score_n, score_z, score_p = calculate_score(lang)
    data = np.array([score_n, score_z, score_p])
    print("Lang is " + lang, ">>>")
    test_list = [wilcoxon, sign_test, permute_test ]
    for i in range(data.shape[0]):
        print("Testing leader is", i, ">>")
        for test in test_list:
            print(test.__name__)
            calculate_Benjamini(data[i], data[i - 1], data[i - 2], test, 0.05, [i, (i - 1)%3, (i - 2)%3])

### Тестируем пару языков "en_ru"

In [253]:
multitest("en_ru")

Lang is en_ru >>>
Testing leader is 0 >>
wilcoxon
Can't reject both hypotheses, for [ 0 , 2 ] -  0.431640625 for [ 0 , 1 ] -  0.322265625
sign_test
Can't reject both hypotheses, for [ 0 , 2 ] -  0.5078125 for [ 0 , 1 ] -  0.7539062500000002
permute_test
Can't reject both hypotheses, for [ 0 , 2 ] -  0.8359375 for [ 0 , 1 ] -  0.115234375
Testing leader is 1 >>
wilcoxon
Can't reject both hypotheses, for [ 1 , 0 ] -  0.322265625 for [ 1 , 2 ] -  0.193359375
sign_test
Can't reject both hypotheses, for [ 1 , 0 ] -  0.7539062500000002 for [ 1 , 2 ] -  0.7539062500000002
permute_test
Can't reject both hypotheses, for [ 1 , 0 ] -  0.884765625 for [ 1 , 2 ] -  0.9423828125
Testing leader is 2 >>
wilcoxon
Can't reject both hypotheses, for [ 2 , 1 ] -  0.193359375 for [ 2 , 0 ] -  0.431640625
sign_test
Can't reject both hypotheses, for [ 2 , 1 ] -  0.7539062500000002 for [ 2 , 0 ] -  0.5078125
permute_test
Can't reject both hypotheses, for [ 2 , 1 ] -  0.0576171875 for [ 2 , 0 ] -  0.1640625


#### Ничего не можем отклонить - нет лидера для  en_ru

## Пара языков kk_ru

In [254]:
multitest("kk_ru") 

Lang is kk_ru >>>
Testing leader is 0 >>
wilcoxon
Can't reject both hypotheses, for [ 0 , 2 ] -  0.625 for [ 0 , 1 ] -  0.921875
sign_test
Can't reject both hypotheses, for [ 0 , 2 ] -  0.5078125 for [ 0 , 1 ] -  0.5078125
permute_test
Can't reject both hypotheses, for [ 0 , 2 ] -  0.5625 for [ 0 , 1 ] -  0.779296875
Testing leader is 1 >>
wilcoxon
Can't reject both hypotheses, for [ 1 , 0 ] -  0.921875 for [ 1 , 2 ] -  0.322265625
sign_test
Can't reject both hypotheses, for [ 1 , 0 ] -  0.5078125 for [ 1 , 2 ] -  0.5078125
permute_test
Can't reject both hypotheses, for [ 1 , 0 ] -  0.22265625 for [ 1 , 2 ] -  0.126953125
Testing leader is 2 >>
wilcoxon
Can't reject both hypotheses, for [ 2 , 1 ] -  0.322265625 for [ 2 , 0 ] -  0.625
sign_test
Can't reject both hypotheses, for [ 2 , 1 ] -  0.5078125 for [ 2 , 0 ] -  0.5078125
permute_test
Can't reject both hypotheses, for [ 2 , 1 ] -  0.875 for [ 2 , 0 ] -  0.4375


#### Ничего не можем отклонить - нет лидера для  kk_ru

### Пара языков en_de

In [255]:
multitest("en_de") 

Lang is en_de >>>
Testing leader is 0 >>
wilcoxon
Can't reject both hypotheses, for [ 0 , 2 ] -  0.76953125 for [ 0 , 1 ] -  0.048828125
sign_test
Can't reject both hypotheses, for [ 0 , 2 ] -  0.7539062500000002 for [ 0 , 1 ] -  0.10937500000000003
permute_test
One hypothesis rejected, for [ 0 , 2 ] -  0.4638671875 for [ 0 , 1 ] -  0.01953125
Testing leader is 1 >>
wilcoxon
Both hypotheses rejected, for [ 1 , 0 ] -  0.048828125 for [ 1 , 2 ] -  0.013671875
sign_test
Can't reject both hypotheses, for [ 1 , 0 ] -  0.10937500000000003 for [ 1 , 2 ] -  0.10937500000000003
permute_test
Can't reject both hypotheses, for [ 1 , 0 ] -  0.98046875 for [ 1 , 2 ] -  0.994140625
Testing leader is 2 >>
wilcoxon
One hypothesis rejected, for [ 2 , 1 ] -  0.013671875 for [ 2 , 0 ] -  0.76953125
sign_test
Can't reject both hypotheses, for [ 2 , 1 ] -  0.10937500000000003 for [ 2 , 0 ] -  0.7539062500000002
permute_test
One hypothesis rejected, for [ 2 , 1 ] -  0.005859375 for [ 2 , 0 ] -  0.5361328125


Получаем, что permute_test с односторонней альтернативой отвергает по одной гипотезе в эксперименте "лидер - это 0(n)" и "лидер - это 2(p)", а Уилкоксон с двусторонней альтернативой отвергает обе гипотезы для "лидер - это 1(z)"
#### Значит, для en_de мы нашли не лидера, а аутсайдера (это z), а про p, n мы ничего сказать не можем

# Другой дизайн эксперимента
Теперь мы тестируем гипотезу о равенстве средних для всех упорядоченных пар(если проверяем одностороннюю альтернативу) или для всех неупорядоченных пар(если проверяем двухстороннюю альтернативу) с поправкой Бенджамини для всего множества получившихся пар.

In [261]:
def calculate_Benjamini(data, test, alpha, is_one_sided=False):
    #data is shape set_number x set_size
    set_len = data.shape[0]
    p_val_list = []
    if is_one_sided:
        for i in range(set_len):
            for j in range(set_len):
                if i != j:
                    _, p_val = test(data[i], data[j], "greater")
                    p_val_list.append([p_val, [i , j]])
    else:
        for i in range(set_len - 1):
            for j in range(i + 1, set_len):
                _, p_val = test(data[i], data[j])
                p_val_list.append([p_val, [i , j]])
    p_val_list = sorted(p_val_list, key= lambda x: x[0])
    m = len(p_val_list)
    for i in range(m):
        if p_val_list[i][0] > alpha/(m - i):
            print("hypotheses from", i ,"accepted")
            break
    return p_val_list

### Тестируем пару en_ru

In [262]:
data = np.array(calculate_score("en_ru"))

In [263]:
calculate_Benjamini(data, permute_test, 0.05, True)

hypotheses from 0 accepted


[[0.0576171875, [2, 1]],
 [0.115234375, [0, 1]],
 [0.1640625, [2, 0]],
 [0.8359375, [0, 2]],
 [0.884765625, [1, 0]],
 [0.9423828125, [1, 2]]]

In [264]:
calculate_Benjamini(data, sign_test, 0.05)

hypotheses from 0 accepted


[[0.5078125, [0, 2]],
 [0.7539062500000002, [0, 1]],
 [0.7539062500000002, [1, 2]]]

In [265]:
calculate_Benjamini(data, wilcoxon, 0.05)

hypotheses from 0 accepted


[[0.193359375, [1, 2]], [0.322265625, [0, 1]], [0.431640625, [0, 2]]]

#### Ничего не можем отклонить - нет лидера для en_ru

### Тестируем пару en_de

In [266]:
data = np.array(calculate_score("en_de"))
## (n = 0, z = 1, p = 2)
calculate_Benjamini(data, permute_test, 0.05, True)

hypotheses from 1 accepted


[[0.005859375, [2, 1]],
 [0.01953125, [0, 1]],
 [0.4638671875, [0, 2]],
 [0.5361328125, [2, 0]],
 [0.98046875, [1, 0]],
 [0.994140625, [1, 2]]]

In [268]:
calculate_Benjamini(data, wilcoxon, 0.05)

hypotheses from 1 accepted


[[0.013671875, [1, 2]], [0.048828125, [0, 1]], [0.76953125, [0, 2]]]

In [269]:
calculate_Benjamini(data, sign_test, 0.05)

hypotheses from 0 accepted


[[0.10937500000000003, [0, 1]],
 [0.10937500000000003, [1, 2]],
 [0.7539062500000002, [0, 2]]]

#### Oтклоняем гипотезу [1, 2]  - то есть среднее p больше, чем среднее z. Однако, гипотеза о том, что p лучше n (гипотеза [2, 0]) не отвергается даже без поправки. Значит, лидер кто-то из p и n, а z - аутсайдер

### Тестируем пару kk_ru

In [270]:
data = np.array(calculate_score("kk_ru"))
calculate_Benjamini(data, permute_test, 0.05, True)

hypotheses from 0 accepted


[[0.126953125, [1, 2]],
 [0.22265625, [1, 0]],
 [0.4375, [2, 0]],
 [0.5625, [0, 2]],
 [0.779296875, [0, 1]],
 [0.875, [2, 1]]]

In [271]:
calculate_Benjamini(data, sign_test, 0.05)

hypotheses from 0 accepted


[[0.5078125, [0, 1]], [0.5078125, [0, 2]], [0.5078125, [1, 2]]]

In [272]:
calculate_Benjamini(data, wilcoxon, 0.05)

hypotheses from 0 accepted


[[0.322265625, [1, 2]], [0.625, [0, 2]], [0.921875, [0, 1]]]

#### Ничего не можем отклонить - нет лидера для kk_ru

## Вывод: мы ничего не можем сказать о лидерах. Для пары языков "en_de" нашли аутсайдера z